# Citibike ML
In this example we use the [Citibike dataset](https://ride.citibikenyc.com/system-data). Citibike is a bicycle sharing system in New York City. Everyday users choose from 20,000 bicycles at 1300 stations around New York City.

To ensure customer satisfaction Citibike needs to predict how many bicycles will be needed at each station. Maintenance teams from Citibike will check each station and repair or replace bicycles. Additionally, the team will relocate bicycles between stations based on predicted demand. The business needs to be able to run reports of how many bicycles will be needed at a given station on a given day.



## Data Engineering
We begin where all ML use cases do: data engineering. In this section of the demo, we will utilize Snowpark's Python client-side Dataframe API to build an **ELT pipeline**.  We will extract the data from the source system (s3), load it into snowflake and add transformations to clean the data before analysis. 

The data engineer has been told that there is historical data going back to 2013 and new data will be made available at the end of each month. 

For this demo flow we will assume that the organization has the following **policies and processes** :   
-**Dev Tools**: The data engineer can develop in their tool of choice (ie. VS Code, IntelliJ, Pycharm, Eclipse, etc.).  Snowpark Python makes it possible to use any environment where they have a python kernel.  For the sake of a demo we will use Jupyter.  
-**Data Governance**: To preserve customer privacy no data can be stored locally.  The ingest system may store data temporarily but it must be assumed that, in production, the ingest system will not preserve intermediate data products between runs. Snowpark Python allows the user to push-down all operations to Snowflake and bring the code to the data.   
-**Automation**: Although the data engineer can use any IDE or notebooks for development purposes the final product must be python code at the end of the work stream.  Well-documented, modularized code is necessary for good ML operations and to interface with the company's CI/CD and orchestration tools.  
-**Compliance**: Any ML models must be traceable back to the original data set used for training.  The business needs to be able to easily remove specific user data from training datasets and retrain models.  

Input: Historical bulk data at `https://s3.amazonaws.com/tripdata/`. Incremental data to be loaded one month at a time.  
Output: `trips` table

In [ ]:
import snowflake.snowpark as snp
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T

import pandas as pd
from datetime import datetime
import requests
from zipfile import ZipFile
from io import BytesIO
import os

### 1. Load  credentials and connect to Snowflake


We will utilize a simple json file to store our credentials. This should **never** be done in production and is for demo purposes only.

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')

### 2. Create a stage for loading data to Snowflake

In [ ]:
load_stage_name=state_dict['load_stage_name']
session.sql('CREATE STAGE IF NOT EXISTS '+load_stage_name).collect()

### 3. Extract:  


Create a list of files to download and upload to stage

In [ ]:
import pandas as pd
from datetime import datetime

#For files like 201306-citibike-tripdata.zip
date_range1 = pd.period_range(start=datetime.strptime("201306", "%Y%m"), 
                             end=datetime.strptime("201612", "%Y%m"), 
                             freq='M').strftime("%Y%m")
file_name_end1 = '-citibike-tripdata.zip'
files_to_download = [date+file_name_end1 for date in date_range1.to_list()]


Starting in January 2017 Citibike changed the format of the file name.

In [ ]:
#For files like 201701-citibike-tripdata.csv.zip
date_range2 = pd.period_range(start=datetime.strptime("201701", "%Y%m"), 
                             end=datetime.strptime("202112", "%Y%m"), 
                             freq='M').strftime("%Y%m")
file_name_end2 = '-citibike-tripdata.csv.zip'
files_to_download = files_to_download + [date+file_name_end2 for date in date_range2.to_list()]

For development purposes we will start with loading just a couple of files.  We will create a bulk load process afterwards.

In [ ]:
files_to_download = [files_to_download[i] for i in [0,102]] #19,50,100,102]]
files_to_download

In [ ]:
session.use_warehouse(state_dict['compute_parameters']['fe_warehouse'])

In [ ]:
schema1_download_files = list()
schema2_download_files = list()
schema2_start_date = datetime.strptime('202102', "%Y%m")

for file_name in files_to_download:
    file_start_date = datetime.strptime(file_name.split("-")[0], "%Y%m")
    if file_start_date < schema2_start_date:
        schema1_download_files.append(file_name)
    else:
        schema2_download_files.append(file_name)

In [ ]:
schema1_download_files, schema2_download_files

In [ ]:
schema1_load_stage = state_dict['load_stage_name']+'/schema1/'
schema2_load_stage = state_dict['load_stage_name']+'/schema2/'

schema1_files_to_load = list()
for zip_file_name in schema1_download_files:
    
    url = state_dict['download_base_url']+zip_file_name
    
    print('Downloading and unzipping: '+url)
    r = requests.get(url)
    file = ZipFile(BytesIO(r.content))
    csv_file_name=file.namelist()[0]
    file.extract(csv_file_name)
    file.close()
    
    print('Putting '+csv_file_name+' to stage: '+schema1_load_stage)
    session.file.put(local_file_name=csv_file_name, 
                     stage_location=schema1_load_stage, 
                     source_compression='NONE', 
                     overwrite=True)
    schema1_files_to_load.append(csv_file_name)
    os.remove(csv_file_name)
    
schema2_files_to_load = list()
for zip_file_name in schema2_download_files:
    
    url = state_dict['download_base_url']+zip_file_name
    
    print('Downloading and unzipping: '+url)
    r = requests.get(url)
    file = ZipFile(BytesIO(r.content))
    csv_file_name=file.namelist()[0]
    file.extract(csv_file_name)
    file.close()
    
    print('Putting '+csv_file_name+' to stage: '+schema2_load_stage)
    session.file.put(local_file_name=csv_file_name, 
                     stage_location=schema2_load_stage, 
                     source_compression='NONE', 
                     overwrite=True)
    schema2_files_to_load.append(csv_file_name)
    os.remove(csv_file_name)

In [ ]:
session.sql("list @"+load_stage_name+" pattern='.*20.*[.]gz'").collect()

### 4. Load: 
Load raw as all string type.  We will fix data types in the transform stage.

There are two schema types so we will create two ingest tables.

In [ ]:
#Upper case fields are common to both schemas.
#Schema from 2013 to 2021
load_schema1 = T.StructType([T.StructField("tripduration", T.StringType()),
                             T.StructField("STARTTIME", T.StringType()), 
                             T.StructField("STOPTIME", T.StringType()), 
                             T.StructField("START_STATION_ID", T.StringType()),
                             T.StructField("START_STATION_NAME", T.StringType()), 
                             T.StructField("START_STATION_LATITUDE", T.StringType()),
                             T.StructField("START_STATION_LONGITUDE", T.StringType()),
                             T.StructField("END_STATION_ID", T.StringType()),
                             T.StructField("END_STATION_NAME", T.StringType()), 
                             T.StructField("END_STATION_LATITUDE", T.StringType()),
                             T.StructField("END_STATION_LONGITUDE", T.StringType()),
                             T.StructField("bike_id", T.StringType()),
                             T.StructField("USERTYPE", T.StringType()), 
                             T.StructField("birth_year", T.StringType()),
                             T.StructField("gender", T.StringType())])

#starting in February 2021 the schema changed
load_schema2 = T.StructType([T.StructField("ride_id", T.StringType()), 
                             T.StructField("rideable_type", T.StringType()), 
                             T.StructField("STARTTIME", T.StringType()), 
                             T.StructField("STOPTIME", T.StringType()), 
                             T.StructField("START_STATION_NAME", T.StringType()), 
                             T.StructField("START_STATION_ID", T.StringType()),
                             T.StructField("END_STATION_NAME", T.StringType()), 
                             T.StructField("END_STATION_ID", T.StringType()),
                             T.StructField("START_STATION_LATITUDE", T.StringType()),
                             T.StructField("START_STATION_LONGITUDE", T.StringType()),
                             T.StructField("END_STATION_LATITUDE", T.StringType()),
                             T.StructField("END_STATION_LONGITUDE", T.StringType()),
                             T.StructField("USERTYPE", T.StringType())])

trips_table_schema = T.StructType([T.StructField("STARTTIME", T.StringType()), 
                             T.StructField("STOPTIME", T.StringType()), 
                             T.StructField("START_STATION_NAME", T.StringType()), 
                             T.StructField("START_STATION_ID", T.StringType()),
                             T.StructField("END_STATION_NAME", T.StringType()), 
                             T.StructField("END_STATION_ID", T.StringType()),
                             T.StructField("START_STATION_LATITUDE", T.StringType()),
                             T.StructField("START_STATION_LONGITUDE", T.StringType()),
                             T.StructField("END_STATION_LATITUDE", T.StringType()),
                             T.StructField("END_STATION_LONGITUDE", T.StringType()),
                             T.StructField("USERTYPE", T.StringType())])

Create empty tables.

In [ ]:
session.createDataFrame([[None]*len(load_schema1.names)], schema=load_schema1)\
       .na.drop()\
       .write\
       .saveAsTable(state_dict['load_table_name']+'schema1')

In [ ]:
session.createDataFrame([[None]*len(load_schema2.names)], schema=load_schema2)\
       .na.drop()\
       .write\
       .saveAsTable(state_dict['load_table_name']+'schema2')

Load schema1 data

In [ ]:
csv_file_format_options = {"FIELD_OPTIONALLY_ENCLOSED_BY": "'\"'", "skip_header": 1}

loaddf = session.read.option("SKIP_HEADER", 1)\
                     .option("FIELD_OPTIONALLY_ENCLOSED_BY", "\042")\
                     .option("COMPRESSION", "GZIP")\
                     .option("NULL_IF", "\\\\N")\
                     .option("NULL_IF", "NULL")\
                     .option("pattern", "'.*20.*[.]gz'")\
                     .schema(load_schema1)\
                     .csv('@'+schema1_load_stage)\
                     .copy_into_table(state_dict['load_table_name']+str('schema1'), 
                                      format_type_options=csv_file_format_options)

Load schema2 data

In [ ]:
loaddf = session.read.option("SKIP_HEADER", 1)\
                     .option("FIELD_OPTIONALLY_ENCLOSED_BY", "\042")\
                     .option("COMPRESSION", "GZIP")\
                     .option("NULL_IF", "\\\\N")\
                     .option("NULL_IF", "NULL")\
                     .option("pattern", "'.*20.*[.]gz'")\
                     .schema(load_schema1)\
                     .csv('@'+schema2_load_stage)\
                     .copy_into_table(state_dict['load_table_name']+str('schema2'), 
                                      format_type_options=csv_file_format_options)

### 5. Transform:
We have the raw data loaded. Now let's transform this data and clean it up. This will push the data to a final \"transformed\" table to be consumed by our Data Science team.  First we start by combining the two tables with the common columns.


In [ ]:
trips_table_schema_names = [field.name for field in trips_table_schema.fields]
transdf1 = session.table(state_dict['load_table_name']+'schema1')[trips_table_schema_names]
transdf2 = session.table(state_dict['load_table_name']+'schema2')[trips_table_schema_names]
transdf = transdf1.union_by_name(transdf2)

There are three different date formats "2014-08-10 15:21:22", "1/1/2015 1:30" and "12/1/2014 02:04:53"

In [ ]:
date_format_2 = "1/1/2015 [0-9]:.*$"      #1/1/2015 1:30 -> #M*M/D*D/YYYY H*H:M*M(:SS)*
date_format_3 = "1/1/2015 [0-9][0-9]:.*$" #1/1/2015 10:30 -> #M*M/D*D/YYYY H*H:M*M(:SS)*
date_format_4 = "12/1/2014.*"             #12/1/2014 02:04:53 -> M*M/D*D/YYYY 

#Change all dates to YYYY-MM-DD HH:MI:SS format
date_format_match = "^([0-9]?[0-9])/([0-9]?[0-9])/([0-9][0-9][0-9][0-9]) ([0-9]?[0-9]):([0-9][0-9])(:[0-9][0-9])?.*$"
date_format_repl = "\\3-\\1-\\2 \\4:\\5\\6"

In [ ]:
transdf.withColumn('STARTTIME', F.regexp_replace(F.col('STARTTIME'),
                                            F.lit(date_format_match), 
                                            F.lit(date_format_repl)))\
      .withColumn('STARTTIME', F.to_timestamp('STARTTIME'))\
      .withColumn('STOPTIME', F.regexp_replace(F.col('STOPTIME'),
                                            F.lit(date_format_match), 
                                            F.lit(date_format_repl)))\
      .withColumn('STOPTIME', F.to_timestamp('STOPTIME'))\
      .select(F.col('STARTTIME'), 
              F.col('STOPTIME'), 
              F.col('START_STATION_ID'), 
              F.col('START_STATION_NAME'), 
              F.col('START_STATION_LATITUDE'), 
              F.col('START_STATION_LONGITUDE'), 
              F.col('END_STATION_ID'), 
              F.col('END_STATION_NAME'), F.col('END_STATION_LATITUDE'), 
              F.col('END_STATION_LONGITUDE'), 
              F.col('USERTYPE'))\
      .write.mode('overwrite').saveAsTable(state_dict['trips_table_name'])

In [ ]:
testdf = session.table(state_dict['trips_table_name'])
testdf.schema

In [ ]:
testdf.count()

### 6. Export code in functional modules for MLOps and orchestration

In [ ]:
%%writefile dags/elt.py
def schema1_definition():
    from snowflake.snowpark import types as T
    load_schema1 = T.StructType([T.StructField("TRIPDURATION", T.StringType()),
                             T.StructField("STARTTIME", T.StringType()), 
                             T.StructField("STOPTIME", T.StringType()), 
                             T.StructField("START_STATION_ID", T.StringType()),
                             T.StructField("START_STATION_NAME", T.StringType()), 
                             T.StructField("START_STATION_LATITUDE", T.StringType()),
                             T.StructField("START_STATION_LONGITUDE", T.StringType()),
                             T.StructField("END_STATION_ID", T.StringType()),
                             T.StructField("END_STATION_NAME", T.StringType()), 
                             T.StructField("END_STATION_LATITUDE", T.StringType()),
                             T.StructField("END_STATION_LONGITUDE", T.StringType()),
                             T.StructField("BIKEID", T.StringType()),
                             T.StructField("USERTYPE", T.StringType()), 
                             T.StructField("BIRTH_YEAR", T.StringType()),
                             T.StructField("GENDER", T.StringType())])
    return load_schema1

def schema2_definition():
    from snowflake.snowpark import types as T
    load_schema2 = T.StructType([T.StructField("ride_id", T.StringType()), 
                             T.StructField("rideable_type", T.StringType()), 
                             T.StructField("STARTTIME", T.StringType()), 
                             T.StructField("STOPTIME", T.StringType()), 
                             T.StructField("START_STATION_NAME", T.StringType()), 
                             T.StructField("START_STATION_ID", T.StringType()),
                             T.StructField("END_STATION_NAME", T.StringType()), 
                             T.StructField("END_STATION_ID", T.StringType()),
                             T.StructField("START_STATION_LATITUDE", T.StringType()),
                             T.StructField("START_STATION_LONGITUDE", T.StringType()),
                             T.StructField("END_STATION_LATITUDE", T.StringType()),
                             T.StructField("END_STATION_LONGITUDE", T.StringType()),
                             T.StructField("USERTYPE", T.StringType())])
    return load_schema2

def conformed_schema():
    from snowflake.snowpark import types as T
    trips_table_schema = T.StructType([T.StructField("STARTTIME", T.StringType()), 
                             T.StructField("STOPTIME", T.StringType()), 
                             T.StructField("START_STATION_NAME", T.StringType()), 
                             T.StructField("START_STATION_ID", T.StringType()),
                             T.StructField("END_STATION_NAME", T.StringType()), 
                             T.StructField("END_STATION_ID", T.StringType()),
                             T.StructField("START_STATION_LATITUDE", T.StringType()),
                             T.StructField("START_STATION_LONGITUDE", T.StringType()),
                             T.StructField("END_STATION_LATITUDE", T.StringType()),
                             T.StructField("END_STATION_LONGITUDE", T.StringType()),
                             T.StructField("USERTYPE", T.StringType())])
    return trips_table_schema

def extract_trips_to_stage(session, files_to_download: list, download_base_url: str, load_stage_name:str):
    import os 
    import requests
    from zipfile import ZipFile
    import gzip
    from datetime import datetime
    from io import BytesIO
    
    schema1_download_files = list()
    schema2_download_files = list()
    schema2_start_date = datetime.strptime('202102', "%Y%m")

    for file_name in files_to_download:
        file_start_date = datetime.strptime(file_name.split("-")[0], "%Y%m")
        if file_start_date < schema2_start_date:
            schema1_download_files.append(file_name)
        else:
            schema2_download_files.append(file_name)
         
        
    schema1_load_stage = load_stage_name+'/schema1/'
    schema1_files_to_load = list()
    for zip_file_name in schema1_download_files:

        url = download_base_url+zip_file_name

        print('Downloading and unzipping: '+url)
        r = requests.get(url)
        file = ZipFile(BytesIO(r.content))
        csv_file_name=file.namelist()[0]
        file.extract(csv_file_name)
        file.close()

        print('Putting '+csv_file_name+' to stage: '+schema1_load_stage)
        session.file.put(local_file_name=csv_file_name, 
                         stage_location=schema1_load_stage, 
                         source_compression='NONE', 
                         overwrite=True)
        schema1_files_to_load.append(csv_file_name)
        os.remove(csv_file_name)

        
    schema2_load_stage = load_stage_name+'/schema2/'
    schema2_files_to_load = list()
    for zip_file_name in schema2_download_files:

        url = download_base_url+zip_file_name

        print('Downloading and unzipping: '+url)
        r = requests.get(url)
        file = ZipFile(BytesIO(r.content))
        csv_file_name=file.namelist()[0]
        file.extract(csv_file_name)
        file.close()

        print('Putting '+csv_file_name+' to stage: '+schema2_load_stage)
        session.file.put(local_file_name=csv_file_name, 
                         stage_location=schema2_load_stage, 
                         source_compression='NONE', 
                         overwrite=True)
        schema2_files_to_load.append(csv_file_name)
        os.remove(csv_file_name)
        
    load_stage_names = {'schema1' : schema1_load_stage, 'schema2' : schema2_load_stage}
    files_to_load = {'schema1': schema1_files_to_load, 'schema2': schema2_files_to_load}

    return load_stage_names, files_to_load
    
def load_trips_to_raw(session, files_to_load:dict, load_stage_names:dict, load_table_name:str):
    from snowflake.snowpark import functions as F
    from snowflake.snowpark import types as T
    from datetime import datetime
    
    csv_file_format_options = {"FIELD_OPTIONALLY_ENCLOSED_BY": "'\"'", "skip_header": 1}

    if len(files_to_load['schema1']) > 0:
        load_schema1 = schema1_definition()
        loaddf = session.read.option("SKIP_HEADER", 1)\
                         .option("FIELD_OPTIONALLY_ENCLOSED_BY", "\042")\
                         .option("COMPRESSION", "GZIP")\
                         .option("NULL_IF", "\\\\N")\
                         .option("NULL_IF", "NULL")\
                         .option("pattern", "'.*20.*[.]gz'")\
                         .schema(load_schema1)\
                         .csv('@'+load_stage_names['schema1'])\
                         .copy_into_table(load_table_name+'schema1', 
                                          format_type_options=csv_file_format_options)
                              
    if len(files_to_load['schema2']) > 0:
        load_schema2 = schema2_definition()
        loaddf = session.read.option("SKIP_HEADER", 1)\
                         .option("FIELD_OPTIONALLY_ENCLOSED_BY", "\042")\
                         .option("COMPRESSION", "GZIP")\
                         .option("NULL_IF", "\\\\N")\
                         .option("NULL_IF", "NULL")\
                         .option("pattern", "'.*20.*[.]gz'")\
                         .schema(load_schema2)\
                         .csv('@'+load_stage_names['schema2'])\
                         .copy_into_table(load_table_name+'schema2', 
                                          format_type_options=csv_file_format_options)
        
    load_table_names = {'schema1' : load_table_name+str('schema1'), 
                         'schema2' : load_table_name+str('schema2')}
                         
    return load_table_names
    
def transform_trips(session, stage_table_names:dict, trips_table_name:str):
    from snowflake.snowpark import functions as F
        
    #Change all dates to YYYY-MM-DD HH:MI:SS format
    date_format_match = "^([0-9]?[0-9])/([0-9]?[0-9])/([0-9][0-9][0-9][0-9]) ([0-9]?[0-9]):([0-9][0-9])(:[0-9][0-9])?.*$"
    date_format_repl = "\\3-\\1-\\2 \\4:\\5\\6"

    trips_table_schema = conformed_schema()
                         
    trips_table_schema_names = [field.name for field in trips_table_schema.fields]
                         
    transdf1 = session.table(stage_table_names['schema1'])[trips_table_schema_names]
    transdf2 = session.table(stage_table_names['schema2'])[trips_table_schema_names]
                         
    transdf = transdf1.union_by_name(transdf2)\
                      .withColumn('STARTTIME', F.regexp_replace(F.col('STARTTIME'),
                                                                F.lit(date_format_match), 
                                                                F.lit(date_format_repl)))\
                      .withColumn('STARTTIME', F.to_timestamp('STARTTIME'))\
                      .withColumn('STOPTIME', F.regexp_replace(F.col('STOPTIME'),
                                                               F.lit(date_format_match), 
                                                               F.lit(date_format_repl)))\
                      .withColumn('STOPTIME', F.to_timestamp('STOPTIME'))\
                      .write.mode('overwrite').saveAsTable(trips_table_name)

    return trips_table_name

In [ ]:
#session.close()

Test the functions

In [ ]:
from dags import elt as ELT
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')

In [ ]:
file_name_end2 = '202102-citibike-tripdata.csv.zip'
file_name_end1 = '201402-citibike-tripdata.zip'
file_name_end3 = '202003-citibike-tripdata.csv.zip'

files_to_download = [file_name_end1]

load_stage_names, files_to_load = ELT.extract_trips_to_stage(session=session, 
                                                            files_to_download=files_to_download, 
                                                            download_base_url=state_dict['download_base_url'], 
                                                            load_stage_name=state_dict['load_stage_name'])

In [ ]:
stage_table_names = ELT.load_trips_to_raw(session=session, 
                                          files_to_load=files_to_load, 
                                          load_stage_names=load_stage_names, 
                                          load_table_name=state_dict['load_table_name'])

In [ ]:
trips_table_name = ELT.transform_trips(session=session, 
                                       stage_table_names=stage_table_names, 
                                       trips_table_name=state_dict['trips_table_name'])

In [ ]:
#session.close()